In [1]:
# Get class data from
#https://www.lakelandcollege.edu/wp-content/mu-plugins/Laker/as/as/acad_prog/gps_model.php?acadprog=CRT.HVAC&catalog_term=22FA
Auto_program_list=['HVC-062','IND-043','HVC-066']
Predict_list=['HVC-068','HVC-070','HVC-072' ]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import seaborn as sns
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#from sklearn.inspection import permutation_importance

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor


In [3]:
# read each sheet of the excel file
d={}    #create blank dictionary to store info in
Semesters=('FA2017','SP2017','FA2018','SP2018','FA2019','SP2019','FA2020','SP2020','FA2021','SP2021', 'FA2022', 'SP2022')
for Semester in Semesters: #Iterate through and store the xlxs files 
    d[Semester] = pd.read_excel('Data/Technology Student Counts.xlsx', sheet_name = Semester[2:]+ Semester[:2])
   

In [4]:
#Lots of space padded data. Sometimes before, sometimes after.
#Get rid of EXCO, EXCL, and stdcnt blank data rows 

values = ['EXCO        ', 'EXCL        ', '       ']
for Semester in Semesters:
    d[Semester] =  d[Semester][ d[Semester]['Location    '].isin(values) == False]
    d[Semester] =  d[Semester][ d[Semester]['  StdCnt'].isin(values) == False]
    #print(Semester,d[Semester])

In [5]:

#*********************************************Step 1********************************

fall_df = pd.DataFrame()                                                 #Create empty Datafram
keep_list=Auto_program_list                                              #Set the files to keep using the Auto_program_list 
for Semester in Semesters:
    d[Semester+'d'] =  d[Semester].drop(['Location    ', 'CrseTypes   ','Depts ', '  Capacity'],
                                        axis=1).set_index('Section Name') #Remove unwanted columns
    d[Semester+'d'] = d[Semester+'d']['  StdCnt'].astype(str).astype(int) #Change StdCnt str to integer
    d[Semester+'d'] = pd.DataFrame(d[Semester+'d'])                       #Create new dataframes

                                                  
    d['dt'+Semester] = d[Semester+'d'].T                                  #transform -rows to columns and vis a versa
            
#*********************************************Step 2********************************   
    pred= d['dt'+Semester]                          #Load a xlsx file 
    test=pred.columns                               #Get just the column names
    
    for i in range(len(test)):
        k =test[i][0:7]                             # removes the section number from course number
        drop=0                                      # resets whether to drop or not 

    if Semester[:2] == 'FA':
        keep_list=Auto_program_list
    else:
        keep_list=Predict_list
    for keep in (keep_list):
            if k in keep:                           #If the class # (k) = a class on the list
                drop = drop+1                       #don't drop
            
    if drop == 0:                                   # If the class number doesn't == to a class on the list 
          
        pred.drop(test[i], axis =1, inplace = True) #drop it.

#*********************************************Step 3********************************  
fall_df=pd.DataFrame()                                   #Make blank DataFrame
size=0                                                   #Set size = 0

c={}                                                     #Make dictionaty
fall_df=pd.DataFrame()                                   #make empty dataframe
for Semester in Semesters[:-3]:                          #iterate through each semester
    #print(d['dtSP'+str(int(Semester[2:])+1)])
    #print(Semester, d['dt'+Semester])
    Courses=[]                                           #create blank Couses list
    sizes=[]                                             #create blank sizes list
    fall_df2=[]                                          #create enpty Fall_df2 list
    if Semester[:2] == 'FA':                             #only do the fall semesters
        for class_num in Auto_program_list:              #look at each individual class in the list
            #print(class_num[:3]+class_num[4:])
            c[class_num[:3]+class_num[4:]] =d['dt'+Semester].filter(regex=class_num,axis=1).sum() #filter out all other classes

            size=0
            for i in range(len(c[class_num[:3]+class_num[4:]])):    #get the number of students in the class (strip '-')
                    size=size+int(c[class_num[:3]+class_num[4:]][i])# add up if there is more than one section 
                    #print(size)
           # print(class_num[:3]+class_num[4:],'=',size)            #set dictionary entry
            c[class_num] =class_num[:3]+class_num[4:],size          #list of class mnumbers
            Courses.append(class_num)             #list of class sizes 
            sizes.append(size)
            
        for class_num in Predict_list:                              #Get the classes I want predictions for too
            #print(class_num)
            c[class_num[:3]+class_num[4:]] =d['dtSP'+str(int(Semester[2:])+1)].filter(regex=class_num,axis=1).sum()
        #d['dtSP'+str(int(Semester[2:])+1)].filter(regex='HVC-072',axis=1).sum()   
            size=0
            for i in range(len(c[class_num[:3]+class_num[4:]])):
                    size=size+int(c[class_num[:3]+class_num[4:]][i])
                    #print(size)
            Courses.append(class_num)
            sizes.append(size)
  
        fall_df2 = pd.DataFrame(list(zip(Courses, sizes)), columns =['Section Name', 'StdCnt']).set_index('Section Name')
        fall_df2 = fall_df2.T                                      #zip the 2 lists together and transpose
        fall_df=fall_df.append(fall_df2)                           #append to the list of fall classes )By year)
        

print('Training Data:')
print(fall_df)                                                     
 


Training Data:
Section Name  HVC-062  IND-043  HVC-066  HVC-068  HVC-070  HVC-072
StdCnt              7       17        8        7        7        7
StdCnt             13       19       12       11       11       11
StdCnt              7       14        8        8        8        8
StdCnt              6       17        7        6        5        6
StdCnt             10       11       10        8        8        8


In [6]:
 
c={}                                                            #Make dictionaty
                                                                #make emptydataframe
Semester ='FA2022'                                              #last semester
#print(d['dtSP'+str(int(Semester[2:])+1)])
#print(Semester, d['dt'+Semester])
Courses=[]                                                      #create blank Couses list
sizes=[]                                                        #create blank sizes list
fall_df2=[]                                                     #create enpty Fall_df2 list
if Semester[:2] == 'FA':                                        #only do the fall semesters
                                
    for class_num in Auto_program_list:                         #look at each individual class in the list
        c[class_num[:3]+class_num[4:]] =d['dt'+Semester].filter(regex=class_num,axis=1).sum() #filter out all other classes
        size=0
        for i in range(len(c[class_num[:3]+class_num[4:]])):    #get the number of students in the class
                size=size+int(c[class_num[:3]+class_num[4:]][i])# add up if there is more than one section 
        c[class_num] =class_num[:3]+class_num[4:],size          
        Courses.append(class_num)                               #list of class numbers
        sizes.append(size)                                      #list of class sizes 
  
    x_test = pd.DataFrame(list(zip(Courses, sizes)), columns =['Section Name', 'StdCnt']).set_index('Section Name')
    x_test = x_test.T                                           #zip the 2 lists together and transpose
print('Data from',Semester,'to make predcition on')            
print(x_test)



Data from FA2022 to make predcition on
Section Name  HVC-062  IND-043  HVC-066
StdCnt              9       14        9


In [7]:

Course_data_df=pd.DataFrame()
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
old_courses=fall_df
Course_data_df=pd.DataFrame()
for course in Predict_list:
    X = old_courses.drop((Predict_list), axis =1)
    y = old_courses[course]
    X_train,  X_test, y_train, y_test = train_test_split(X,y, test_size = .125)
    grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
    grid_GBR.fit(X_train, y_train)  

    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

    train_mse = mean_squared_error(y_train, grid_GBR.predict(X_train))
    test_mse = mean_squared_error(y_test, grid_GBR.predict(X_test))         
    cd_df=pd.DataFrame({'Course':[course], "train_mse":[train_mse], "test_mse":[test_mse], "predicted":[round(grid_GBR.predict(x_test)[0])]})
    Course_data_df=Course_data_df.append(cd_df)
print(Course_data_df)
print('Based on these fall semester class numbers')
print(x_test)


 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.03, max_depth=6, n_estimators=1500,
                          subsample=0.9)

 The best score across ALL searched params:
 -6.962217184913266

 The best parameters across ALL searched params:
 {'learning_rate': 0.03, 'max_depth': 6, 'n_estimators': 1500, 'subsample': 0.9}
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.02, max_depth=4, n_estimators=1000,
                          subsample=0.9)

 The best score across ALL searched params:
 -0.012271277627553423

 The best parameters across ALL searched params:
 {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.9}
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.03, max_depth=6, n_estimators=500,
                          subsample=0.9)

 The best score a